In [ ]:
import edgar as et
import datetime
from rich.console import Console
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from edgar.xbrl.xbrl import XBRL
from edgar.xbrl import XBRLS

console = Console()

In [ ]:
et.set_identity("lng1492@gmail.com")

# Getting filings

In [ ]:
ticker = "AAPL"
company = et.Company(ticker)
filings = company.get_filings()
filing_df = filings.to_pandas()

summary = {
    "ticker": company.tickers,
    "name": company.name,
    "industry": company.industry,
    "mailing_address": str(company.mailing_address()),
    "cik": company.cik,
    "sic": company.sic,
    "exchanges": company.get_exchanges(),
    "latest_filing_date": filing_df.filing_date.max().strftime("%Y-%m-%d"),
    "earliest_filing_date": filing_df.filing_date.min().strftime("%Y-%m-%d"),
    "form_counts": filing_df.form.value_counts().to_dict() # Dict[str, int]
}

# edgartools/edgar/ownership/ownershipforms.py
#  Form3,4,5 where Ownership class can be converted to Form3,4,5 class using .obj()
#  From here I think we can access the information between date ranges also by filtering and looping over Form filings
#       converting using .obj() then getting activities, holdings, etc.


In [ ]:
filtered_filings[0].

In [ ]:
filtered_filings = filings.filter(form="10-K")
merged_df = [filing..to_dataframe() for filing in filtered_filings]
merged_df = pd.concat(merged_df)

In [ ]:
start_date = merged_df.Date.min()
end_date = merged_df.Date.max() + pd.Timedelta(days=1) # yf does not include the last day
stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data = stock_data.reset_index()
stock_data.columns = stock_data.columns.droplevel(level=1)

In [ ]:
filtered_filings = filings.filter(form="4", date="2024-01-01:")
merged_df = [filing.obj().to_dataframe() for filing in filtered_filings]
merged_df = pd.concat(merged_df)

In [ ]:
# Error in get_statements_impl for
# ticker=AAPL, 
# form=10-K, 
# date=:2024-12-31, 
# statement=IncomeStatement: 
# 'NoneType' object has no attribute 'reporting_periods'"

ticker = "AAPL"
form = "10-K"
date = "2020-01-01:2025-12-31"
statement = "IncomeStatement"
company = et.Company(ticker)
filings = company.get_filings()
xbrls = XBRLS.from_filings(filings.filter(form=form, date=date))

In [ ]:
xbrls.statements['BalanceSheet'].to_dataframe()

In [ ]:
x = xbrls.statements['SegmentDisclosure'].render().to_markdown()

In [ ]:
statement_type = 'BalanceSheet'
statements = xbrls.statements
stitched_statement = statements[statement_type]

found_stmt_types = set()
found_periods = xbrls.get_periods()
for xbrl in stitched_statement.xbrls.xbrl_list:
    statement = xbrl.get_all_statements()
    for stmt in statement:
        if stmt['type']:
            found_stmt_types.add(stmt['type'])
period_count = len(found_periods)
assert period_count > 0 and len(found_stmt_types) > 0

In [ ]:
comprehensive_income = xbrl.statements["StatementOfEquity"]

In [ ]:
type(comprehensive_income)

In [ ]:
xbrl.statements['IncomeStatement']

In [ ]:
filings = company.get_filings()
# filings.get_filings(2024, [3, 4])

In [ ]:
company.get_filings().filter(date="2025-03-01:")

In [ ]:
import datetime
company_insider_filings = company.get_filings(form=[3,4,5])
company_insider_filings.filter

In [ ]:
company_insider_filings = company.get_filings(form=[3,4,5])
company_insider_filings[0].obj() # prints additional information (basically renders the form)
# These two produce the same output
print(company_insider_filings[0].obj())
# console.print(rklb_insider_filings[0].obj().get_ownership_summary())

print(f"rklb_insider_filings[0] type: {type(company_insider_filings[0])}")
print(f"rklb_insider_filings[0].obj() type: {type(company_insider_filings[0].obj())}")

In [ ]:
rklb_filings = rklb.get_filings() # get all filings
# rklb_filings[0].open() # opens browser to the filing
rklb_10q = rklb.get_filings(form='10-Q') 

# rklb_filings.data # pyarrow table of filings

In [ ]:
rklb_10q[0].accession_number

In [ ]:
rklb.get_filings(form='3')[3].view()

# Facts Metadata

In [ ]:
rklb_facts = rklb.get_facts()
rklb_facts_df = rklb_facts.to_pandas() # converts pyarrow table to pandas df

## Gets metadata about the facts including fact description
unique_desc = rklb_facts.fact_meta['description'].value_counts()
# for desc in unique_desc[:3]:
#     print(desc)

In [ ]:
unique_desc

# Attachments in a filing

In [ ]:
# Attachments
## Get the attachments for the first 10q filing
rklb_10q[0].attachments

## Prints the actual text of the attachment 10q
# text = rklb_10q[0].attachments[1].text()

## Atleast for the 10q attachments, I cant find anything useful
# rklb_10q[0].attachments[84].download('.') # download the attachment

# Financials


Some functionality is deprecated, should use XBRLs instead of i.e. MultiFinancials(filings)

In [ ]:
rklb_financials = rklb.get_financials()

In [ ]:
# Income Statement
rklb_balance_sheet = rklb_financials.balance_sheet()
rklb_balance_sheet = rklb_balance_sheet.to_dataframe()

rklb_cash_flow = rklb_financials.cashflow_statement()
rklb_cash_flow = rklb_cash_flow.to_dataframe()

rklb_income_statement = rklb_financials.income_statement()
rklb_income_statement = rklb_income_statement.to_dataframe()

In [ ]:
from edgar.xbrl.xbrl import XBRL
from edgar.xbrl import XBRLS

filing = rklb.latest("10-K")
xbrl = XBRL.from_filing(filing)
rklb_income_statement = xbrl.statements.income_statement()

In [ ]:
filings = rklb.latest("10-K", 5)
xbrls = XBRLS.from_filings(filings)
stitched_statements = xbrls.statements

stitched_statements

In [ ]:
balance_sheet = stitched_statements.balance_sheet()
income_statement = stitched_statements.income_statement()
cash_flow = stitched_statements.cashflow_statement()
statement_of_equity = stitched_statements.statement_of_equity()

# You can also access by type
comprehensive_income = stitched_statements["ComprehensiveIncome"]

# income_statement.to_dataframe() # converts to pandas df

In [ ]:
income_trend = stitched_statements.income_statement(max_periods=3)

In [ ]:
from rich.console import Console
console = Console()
console.print(stitched_statements.balance_sheet())

# Insider Trading

- Form 3: Filed by insiders to report their initial ownership of company stock - typically filed when an insider joins a company or becomes an officer or director.
- Form 4: Filed to report any changes in ownership of company stock - typically filed when an insider buys or sells company stock.
- Form 5: Includes any transactions that were not reported on Form 4 - typically filed at the end of the fiscal year.

NOTE: A "filing" is returned as an `edgar.entity.filings.EntityFiling` object. We have to use `.obj()` to convert it get something useful like form4 `edgar.ownership.ownershipforms.Form4`

In [ ]:
rklb_insider_filings = rklb.get_filings(form=[3,4,5])

In [ ]:
rklb_insider_filings[0].obj() # prints additional information (basically renders the form)

In [ ]:
# These two produce the same output
console.print(rklb_insider_filings[0].obj())
# console.print(rklb_insider_filings[0].obj().get_ownership_summary())

console.print(f"rklb_insider_filings[0] type: {type(rklb_insider_filings[0])}")
console.print(f"rklb_insider_filings[0].obj() type: {type(rklb_insider_filings[0].obj())}")